# Travel Agent

This notebook implements a travel agent AI using **LangChain Agent**, **MySQL MCP Server**, **HeatWave** database, and **OCI LLMs** for hotel bookings, enabling seamless travel planning through natural conversation.  

Example use cases are:
- Search for hotels  
- Display room options  
- Make reservations  

This notebook requires **Python version >=3.12**, with required packages listed below: <br>
**langchain langgraph langchain-core langchain_oci langchain_mcp_adapters nest_asyncio oci fastapi fastmcp uvicorn**

The notebook consists of the following parts: <br>

<ol>
  <li>Setup
    <ul>
      <li>Imports</li>
      <li>Resource Configuration (you need to provide necessary information here)</li>
      <li>Global Rules</li>
    </ul>
  </li>
  <li>Utility Functions</li>
  <li>Core Function</li>
  <li>Examples</li>
</ol>

## Setup

### Imports

In [ ]:
import ast
import asyncio
import uuid
import time

from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.callbacks.base import BaseCallbackHandler
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_oci import ChatOCIGenAI

# Enable running async function in Juypter environment
import nest_asyncio
nest_asyncio.apply()

### Resource Configuration

#### OCI LLM
The LLM that orchestrates the workflow of the travel agent.

In [ ]:
# Replace the following variables with your own values to enable access to OCI LLMs.
OCI_AUTH_PROFILE = "<YOUR_OCI_AUTH_PROFILE>"    # defined in ~/.oci/config
OCI_COMPARTMENT_ID = "<YOUR_OCI_COMPARTMENT_ID>"
 

OCI_MODEL_ID = "xai.grok-4" # You may change it to your preferred model.
llm = ChatOCIGenAI(
    model_id=OCI_MODEL_ID,
    auth_profile=OCI_AUTH_PROFILE,
    compartment_id=OCI_COMPARTMENT_ID,
    model_kwargs={"temperature": 0, "top_k": 1},
)

#### MCP Server
The MySQL MCP Server that provides tools needed by the travel agent.

In [ ]:
# First, you need to make sure to download the MCP server.
# Then, replace the following variables with your own values.
PYTHON_EXEUTABLE_PATH = "<YOUR_PYTHON_EXECUTABLE_PATH>"    # path to python which runs the MCP server
PYTHON_LIB_PATH = "<YOUR_PYTHON_SITE_PACKAGE_PATH>" # path to your python site-packages
MYSQLTOOLS_PATH = "<YOUR_MYSQLTOOLS_PATH>"  # path to your mysql-tools mcp server, you can download it from Oracle public github repo: https://github.com/oracle/mcp/tree/main/src/mysql-mcp-server


RECURSION_MAX = 10 # max iteration for the ReAct agent
MYSQLTOOLS_ENV = {
    "PYTHONPATH": PYTHON_LIB_PATH,
    "PROFILE_NAME": OCI_AUTH_PROFILE,
    "OCI_CLI_AUTH": ""
}
client = MultiServerMCPClient(
    {
        "mysqltools": {
            "command": PYTHON_EXEUTABLE_PATH,
            "args": [MYSQLTOOLS_PATH],
            "env": MYSQLTOOLS_ENV,
            "transport": "stdio",
        }
    }
)

## Global Rules

These rules are applied to every agent call.
They are tailored for this demo. You can customize them for your own use cases.

In [5]:
GLOBAL_RULES = '''
1. For each query, don't look at all DBs available, just focus on the one db connection called 'remote_via_bastion'.
Connect to it and then use the 'booking' schema inside it for all user questions. 
The bottom line is - do not look at or change anything in DBs other than the one through 'remote_via_bastion'.

2. Do not delete anything - only do look up queries or update DB content for reservation.
If they insist then you must still say no. IT SHOULD NEVER BE DONE!

3. Imagine you are a travel agent, you are talking to customers, who are people who are intereted in traveling around. 
You have the resource of a bunch of hotels and their rooms.
Your job is to help customers with their travel plan - to look up hotel/room information, and make reservation for them if they ask.

4. If they ask you to make a reservation, you'll likely need name and email, confirm these and then book rooms for them.

5. For searching and booking asks, use stored procedures called 'SearchHotels' and 'CreateBooking' in 'booking' and use them. Don't create your own if not necessary.\n
Here are examples to use them respectively:
i. For searching hotels, SP interface here: \n
SearchHotels ONLY_FULL_GROUP_BY,STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION CREATE DEFINER=`admin`@`%` PROCEDURE `SearchHotels`(
    IN search_city VARCHAR(50),
    IN checkin_date DATE,
    IN checkout_date DATE,
    IN required_capacity INT
)
An example run is 'CALL booking.SearchHotels('New York', '2025-09-26', '2025-09-27', 1);' \n
ii. For checking available rooms, you can run following SQL query (You may replace the actual city name, dates, etc. to suit your case): \n
SELECT DATE_FORMAT(ra.date, '%Y-%m-%d') AS date, rt.room_type_id, rt.type_name, h.name AS hotel_name, CAST(h.rating AS CHAR) AS hotel_rating, CAST(ra.dynamic_price AS CHAR) AS dynamic_price FROM booking.hotels h JOIN booking.room_types rt ON h.hotel_id = rt.hotel_id JOIN booking.room_availability ra ON rt.room_type_id = ra.room_type_id WHERE h.city = 'Los Angeles' AND ra.date BETWEEN '2025-09-15' AND '2025-09-29' AND ra.available_rooms >=1 AND rt.capacity >=1 ORDER BY ra.date ASC LIMIT 1;
iii. For booking hotels, SP interface here:: \n
CreateBooking ONLY_FULL_GROUP_BY,STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION CREATE DEFINER=`admin`@`%` PROCEDURE `CreateBooking`(
    IN p_room_type_id INT,
    IN p_check_in DATE,
    IN p_check_out DATE,
    IN p_guest_name VARCHAR(100),
    IN p_guest_email VARCHAR(100),
    IN p_rooms_needed INT,
    OUT p_booking_id INT,
    OUT p_total_price DECIMAL(10,2),
    OUT p_status VARCHAR(50)
)
An example run is 'CALL booking.CreateBooking(3, '2025-09-15', '2025-09-16', 'Guest_456', 'guest_456@gmail.com', 1, @p_booking_id, @p_total_price, @p_status);' \n
After using CreatingBooking, you can run 'SELECT @booking_id AS booking_id, CAST(@total_price AS CHAR) AS total_price, @status AS status;' to confirm.

6. Keep your responses friendly yet not too lengthy - be a nice and efficient agent.

7. Don't make repeated reservation.

8. If there are duplicated information in the answer, hide the duplicated part. But you don't need to mention "we have duplicates..." explicitly to users.

9. The default year is 2025 if not explicitly specified.

11. If you need to run a tool then run it, don't just respond with an unrun tool call.

12. When crafting SELECT queries on tables with decimal columns (e.g., rating in hotels), always cast them to CHAR to ensure results are returned properly without errors.

13. When crafting SELECT queries on tables with date or datetime columns, always use DATE_FORMAT to convert them to strings (e.g., DATE_FORMAT(date_col, '%Y-%m-%d') for dates, DATE_FORMAT(datetime_col, '%Y-%m-%d %H:%i:%s') for datetimes) to avoid serialization issues.

14. After receiving successful query results from a tool (not an error), use the data to formulate your final plain text response. Do not call the tool again unless the output indicates an error.

15. Don't run same SQL query repeatedly unless you are checking recent - if you are running something you have run before, just grab the result directly.
'''

## Utility Functions

In [ ]:
# Send request to the agent, which will be handled asynchrounously.
def agent_call(agent, messages, thread_id, callbacks=None):
    config = {
        "configurable": {"thread_id": thread_id},
        "recursion_limit": RECURSION_MAX,
    }
    if callbacks:
        config["callbacks"] = callbacks
    all_messages = [("system", GLOBAL_RULES)] + list(messages)
    return (thread_id, asyncio.run(agent.ainvoke({"messages": all_messages}, config=config)))


# Callback handler - prints the agent's status at the start and end of LLM calls, tool calls, etc.
# Used to track the agent's progress.
class MilestoneProgressHandler(BaseCallbackHandler):
    def __init__(self, user_intent=None):
        self._first_chain_logged = False
        self._last = None

    def _print(self, msg):
        if msg != self._last:
            print(f"[{time.strftime('%H:%M:%S')}] {msg}")
            self._last = msg

    def on_chain_start(self, serialized, inputs, **kwargs):
        if not self._first_chain_logged:
            self._print(f"Starting agent ⚙️")
            self._first_chain_logged = True

    def on_chain_error(self, error, **kwargs):
        self._print(f"Agent error: {error}")

    def on_llm_start(self, serialized, prompts, **kwargs):
        self._print(f"Thinking... 🧠")

    def on_llm_end(self, response, **kwargs):
        pass

    def on_llm_error(self, error, **kwargs):
        self._print(f"LLM error: {error}")

    def on_tool_start(self, serialized, input_str, **kwargs):
        name = (serialized or {}).get("name") or "tool"
        friendly = {
            "execute_sql_tool_by_connection_id": "Running database query... 🔍",
        }.get(name, f"Using tool... 🛠️: **{name}**")
        self._print(f"{friendly}")
        
        if name == "execute_sql_tool_by_connection_id":
            sql = ast.literal_eval(input_str).get("sql_script", "")
            self._print("SQL Statement:\n" + sql)

    def on_tool_end(self, output, **kwargs):
        self._print("finished using tool ✅")

    def on_tool_error(self, error, **kwargs):
        self._print(f"Tool error: {error}")
        
    
# This function helps extract the AI agent response from its JSON output.
def extract_ai_response(resp):
    msgs = resp.get("messages") or []
    for msg in reversed(msgs):
        role = getattr(msg, "type", None)
        if role in ("ai", "assistant"):
            return getattr(msg, "content", None)

## Core Function
The function defines the agent’s behavior in response to a user request, utilizing the utility functions above.

In [ ]:
# Create a ReAct agent to interact with the user
tools = await client.get_tools()
agent = create_react_agent(llm, tools, checkpointer=MemorySaver())
thread_id = str(uuid.uuid1())

# define example prompts and corresponding input context for the agent
PROMPT_HOTELS = "List all hotels"
PROMPT_ROOMS_LA = "List all rooms in Los Angeles"
PROMPT_RESERVE_LA = "Reserve a room in Los Angeles for a random guest"
PROMPT_RECENT_BOOKINGS = "Check recent booking"

HINT_HOTELS = '''Use 'execute_sql_tool_by_connection_id' to run this SQL query 'SELECT hotel_id, name, city, CAST(rating AS CHAR) AS rating, amenities FROM booking.hotels;' to find all the hotels, then list them in a nicely formatted way.'''

HINT_ROOMS_LA = '''Use 'execute_sql_tool_by_connection_id' to run this SQL query:
SELECT h.hotel_id, h.name AS Hotel, rt.type_name AS Room_Type FROM booking.hotels h JOIN booking.room_types rt ON h.hotel_id = rt.hotel_id WHERE h.city = 'Los Angeles' ORDER BY h.hotel_id, rt.type_name;
to find all the hotels, then list them in a nicely formatted way.'''

HINT_RESERVE_LA = (
    "Reserve a room (one night) for Guest (name) with guest@gmail.com (email) in Los Angeles.\n"
    "Search 9/1 first, if no available room then go to next day, and so on.\n"
    "Follow the workflow below\n"
    "1. SELECT MIN(DATE_FORMAT(ra.date, '%Y-%m-%d')) AS earliest_date FROM booking.room_availability ra JOIN booking.room_types rt ON ra.room_type_id = rt.room_type_id JOIN booking.hotels h ON rt.hotel_id = h.hotel_id WHERE h.city = 'Los Angeles' AND ra.date >= '2025-09-01' AND ra.available_rooms >=1 AND rt.capacity >=1\n"
    "2. SELECT DATE_FORMAT(ra.date, '%Y-%m-%d') AS date, rt.room_type_id, rt.type_name, h.name AS hotel_name, CAST(h.rating AS CHAR) AS hotel_rating, CAST(ra.dynamic_price AS CHAR) AS dynamic_price FROM booking.hotels h JOIN booking.room_types rt ON h.hotel_id = rt.hotel_id JOIN booking.room_availability ra ON rt.room_type_id = ra.room_type_id WHERE h.city = 'Los Angeles' AND ra.date = '<date_with_available_room_you_found_in_step_1>' AND ra.available_rooms >= 1 AND rt.capacity >= 1 ORDER BY ra.dynamic_price ASC\n"
    "3. do this call once and only once to book - CALL booking.CreateBooking('<room_type_id>', '<start_date>', '<end_date>', 'Guest', 'guest@gmail.com', 1, @p_booking_id, @p_total_price, @p_status); SELECT @p_booking_id AS booking_id, @p_total_price AS total_price, @p_status AS status; SELECT @booking_id AS booking_id, CAST(@total_price AS CHAR) AS total_price, @status AS status;\n"
)

HINT_RECENT_BOOKINGS = (
    "Use 'execute_sql_tool_by_connection_id' to run the below SQL query:\n"
    "USE booking; SELECT booking_id, room_type_id, DATE_FORMAT(check_in, '%Y-%m-%d') AS check_in, DATE_FORMAT(check_out, '%Y-%m-%d') AS check_out, guest_name, guest_email, rooms_booked, CAST(total_price AS CHAR) AS total_price, DATE_FORMAT(booking_date, '%Y-%m-%d %H:%i:%s') AS booking_date, status FROM bookings ORDER BY booking_id DESC LIMIT 5\n"
    "to check most recent bookings."
)


# the core function to chat with agent
def chat_with_agent(user_input):
    if not user_input:
        return
    
    handler = MilestoneProgressHandler(user_intent=user_input)
    print(f"[{time.strftime('%H:%M:%S')}] Run started 🚅")
    session_thread_id = thread_id
    
    # apply hint to example prompts
    prompt_to_hint = {
        PROMPT_HOTELS: HINT_HOTELS,
        PROMPT_ROOMS_LA: HINT_ROOMS_LA,
        PROMPT_RESERVE_LA: HINT_RESERVE_LA,
        PROMPT_RECENT_BOOKINGS: HINT_RECENT_BOOKINGS
    }
    agent_input = prompt_to_hint.get(user_input, user_input)

    result_holder = {}
    start_time = time.time()
    messages = [("user", agent_input)]
    
    # calling agent
    thread_id_out, out = agent_call(agent, messages, session_thread_id, callbacks=[handler])
    
    result_holder["thread_id"] = thread_id_out
    result_holder["out"] = out
    elapsed = time.time() - start_time
    print(f"[{time.strftime('%H:%M:%S')}] Run finished in {elapsed:.1f}s")
    out = result_holder.get("out", {})
    response_text = extract_ai_response(out)

    print(f"Assistant: {response_text}")

## Examples
Example execution of the agent chat function, showcasing the agentic workflow — including reasoning and tool usage.

In [8]:
chat_with_agent("List all hotels")

User: List all hotels
[13:59:39] Run started 🚅
[13:59:39] Starting agent ⚙️
[13:59:39] Thinking... 🧠
[13:59:45] Running database query... 🔍
[13:59:45] SQL Statement:
SELECT hotel_id, name, city, CAST(rating AS CHAR) AS rating, amenities FROM booking.hotels;
[13:59:47] finished using tool ✅
[13:59:47] Thinking... 🧠
[13:59:53] Run finished in 14.4s
Assistant: Here's a list of all the hotels available in our system, formatted for easy reading. I've included key details like ID, name, city, rating, and amenities. If you're interested in booking or more info on any, just let me know!

- **ID 1: Grand Plaza Hotel** (New York) - Rating: 4.5 - Amenities: wifi, pool, gym, parking
- **ID 2: Budget Inn** (Los Angeles) - Rating: 3.2 - Amenities: wifi, parking
- **ID 3: Luxury Resort** (Miami) - Rating: 4.8 - Amenities: wifi, pool, spa, beach, restaurant
- **ID 4: Business Center Hotel** (Chicago) - Rating: 4.1 - Amenities: wifi, gym, business_center, parking
- **ID 5: Mountain View Lodge** (Denver

In [9]:
chat_with_agent("List all rooms in Los Angeles")

User: List all rooms in Los Angeles
[13:59:53] Run started 🚅
[13:59:53] Starting agent ⚙️
[13:59:53] Thinking... 🧠
[14:00:03] Running database query... 🔍
[14:00:03] SQL Statement:
SELECT h.hotel_id, h.name AS Hotel, rt.type_name AS Room_Type FROM booking.hotels h JOIN booking.room_types rt ON h.hotel_id = rt.hotel_id WHERE h.city = 'Los Angeles' ORDER BY h.hotel_id, rt.type_name;
[14:00:05] finished using tool ✅
[14:00:05] Thinking... 🧠
[14:00:13] Run finished in 19.3s
Assistant: Here's a nicely formatted list of hotels in Los Angeles, along with their available room types, based on our database:

- **Budget Inn** (ID: 2)  
  - Room Types: Deluxe, Economy  

If you'd like more details, availability for specific dates, or to make a booking, just let me know!


In [10]:
chat_with_agent("Reserve a room in Los Angeles for a random guest")

User: Reserve a room in Los Angeles for a random guest
[14:00:13] Run started 🚅
[14:00:13] Starting agent ⚙️
[14:00:13] Thinking... 🧠
[14:00:25] Running database query... 🔍
[14:00:25] SQL Statement:
SELECT MIN(DATE_FORMAT(ra.date, '%Y-%m-%d')) AS earliest_date FROM booking.room_availability ra JOIN booking.room_types rt ON ra.room_type_id = rt.room_type_id JOIN booking.hotels h ON rt.hotel_id = h.hotel_id WHERE h.city = 'Los Angeles' AND ra.date >= '2025-09-01' AND ra.available_rooms >=1 AND rt.capacity >=1
[14:00:27] finished using tool ✅
[14:00:27] Thinking... 🧠
[14:00:35] Running database query... 🔍
[14:00:35] SQL Statement:
SELECT DATE_FORMAT(ra.date, '%Y-%m-%d') AS date, rt.room_type_id, rt.type_name, h.name AS hotel_name, CAST(h.rating AS CHAR) AS hotel_rating, CAST(ra.dynamic_price AS CHAR) AS dynamic_price FROM booking.hotels h JOIN booking.room_types rt ON h.hotel_id = rt.hotel_id JOIN booking.room_availability ra ON rt.room_type_id = ra.room_type_id WHERE h.city = 'Los Angele

In [11]:
chat_with_agent("Check recent booking")

User: Check recent booking
[14:00:46] Run started 🚅
[14:00:46] Starting agent ⚙️
[14:00:46] Thinking... 🧠
[14:00:52] Running database query... 🔍
[14:00:52] SQL Statement:
USE booking; SELECT booking_id, room_type_id, DATE_FORMAT(check_in, '%Y-%m-%d') AS check_in, DATE_FORMAT(check_out, '%Y-%m-%d') AS check_out, guest_name, guest_email, rooms_booked, CAST(total_price AS CHAR) AS total_price, DATE_FORMAT(booking_date, '%Y-%m-%d %H:%i:%s') AS booking_date, status FROM bookings ORDER BY booking_id DESC LIMIT 5
[14:00:54] finished using tool ✅
[14:00:54] Thinking... 🧠
[14:00:59] Run finished in 13.1s
Assistant: Here's a list of the 5 most recent bookings in our system, formatted for clarity:

- **Booking ID 236** (Room Type ID: 3)  
  Check-in: 2025-09-05 | Check-out: 2025-09-06  
  Guest: Guest (guest@gmail.com) | Rooms: 1 | Total: $121.60  
  Booked on: 2025-11-07 22:00:43 | Status: confirmed

- **Booking ID 235** (Room Type ID: 3)  
  Check-in: 2025-09-05 | Check-out: 2025-09-06  
  Gues

In [12]:
# chat_with_agent("<try_your_own_prompt>")